# Segmenting and Clustering Toronto neighborhoods
This notebook belongs to the assignment (week 3) of IBM's Applied Data Science capstone.

## Scrape the _Wikipedia_ webpage
I have struggled quite some time with scraping the _Wikipedia_ webpage, but fortunately I stumbled upon the <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/"> Beautiful Soup</a> library. This library can be used to pull data from HTML and XML files.

In [1]:
# import Python libraries for web scraping
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

We define the URL to the Wikipedia page, and use the __*requests*__ library to convert the contents of the page into a string. Print the output to the screen reveals that we are dealing with an HTML-formatted page.

In [2]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_page = requests.get(wiki_url)
wiki_text = wiki_page.text
print(wiki_text[:300])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<


Use the __*BeautifulSoup*__ library to instantiate an object from the the HTML-formatted _wikitext_.

In [3]:
soup = BeautifulSoup(wiki_text, 'html.parser')

On the Wiki page, our data of interest is structured in the form of a table. Therefore we look for this table, and then use its contents to instantiate a __*pandas*__ dataframe.

In [4]:
table_of_interest = str(soup.find("table", {"class": "wikitable sortable"}))

In [5]:
df_orig = pd.read_html(table_of_interest)[0]
df_orig.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Clearly the dataframe is not ready yet. We want to update the column headers, and then get rid of the first record (first row).

In [6]:
df_orig.columns = ["PostalCode", "Borough", "Neighborhood"]
df_orig.drop(df_orig.index[0], axis = 0, inplace = True)
df_orig.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Next, we want to remove all records where the _Borough_ is _Not assigned_.

In [7]:
df_orig = df_orig[df_orig["Borough"] != "Not assigned"]
df_orig.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


Now let's check if there also are *Neighborhood*s in our dataset that equal 'Not assigned'.

In [8]:
check_df = df_orig["Neighborhood"] == "Not assigned"
check_df.value_counts()

False    211
True       1
Name: Neighborhood, dtype: int64

Apparently there is one record in our dataset for which the *Neighborhood* is 'Not assigned'. Let's look up this record.

In [9]:
df_orig.loc[df_orig["Neighborhood"] == "Not assigned"]

,PostalCode,Borough,Neighborhood
9,M7A,Queen's Park,Not assigned


To replace the *Neighborhood* that has value 'Not assigned', use the **.loc** and select the right columns.

In [10]:
df_orig.loc[df_orig["Neighborhood"] == "Not assigned", "Neighborhood"] = df_orig.loc[df_orig["Neighborhood"] == 'Not assigned', "Borough"]
df_orig.head(10)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


Use the <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html?highlight=groupby#pandas.DataFrame.groupby">**groupby**</a> and <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.agg.html">**agg**</a> method to aggregate the Neighborhoods that are in the same Borough.

In [11]:
df = df_orig.groupby(["PostalCode"]).agg({"Borough": 'first', "Neighborhood": ", ".join})
df.reset_index(inplace = True)
df.head()

,PostalCode,Neighborhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough


In [15]:
df.shape

(103, 3)

## Get the latitude and longitude for each postal code

We are using the provided .csv file to get the latitude and longitude values for each postal code.

In [12]:
geo_url = "http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
df_coord = pd.read_csv(geo_url)
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Next we update the column headers such that we can merge the two dataframes.

In [14]:
df_coord.columns = ["PostalCode", "Latitude", "Longitude"]
df_full = pd.merge(df, df_coord, on = "PostalCode")
df_full.head()

,PostalCode,Neighborhood,Borough,Latitude,Longitude
0,M1B,"Rouge, Malvern",Scarborough,43.806686,-79.194353
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,43.784535,-79.160497
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,43.763573,-79.188711
3,M1G,Woburn,Scarborough,43.770992,-79.216917
4,M1H,Cedarbrae,Scarborough,43.773136,-79.239476


## Visualize neighborhoods in Toronto

Use the <a href="https://geopy.readthedocs.io/en/stable/">geopy</a> library to get the latitude and longitude of Toronto.

In [28]:
df_toronto = df_full[df_full["Borough"].str.contains("Toronto")].reset_index(drop = True)
print(df_toronto.shape)
df_toronto.head()

(38, 5)


,PostalCode,Neighborhood,Borough,Latitude,Longitude
0,M4E,The Beaches,East Toronto,43.676357,-79.293031
1,M4K,"The Danforth West, Riverdale",East Toronto,43.679557,-79.352188
2,M4L,"The Beaches West, India Bazaar",East Toronto,43.668999,-79.315572
3,M4M,Studio District,East Toronto,43.659526,-79.340923
4,M4N,Lawrence Park,Central Toronto,43.728020,-79.388790


In [16]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00   1.09 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00   1.69 MB/s


In [17]:
search_address = "Toronto"

geo_locator = Nominatim()
location = geo_locator.geocode(search_address)
lat, lng = location.latitude, location.longitude
print("The geographical coordinate of Toronto are {}, {}.".format(lat, lng))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geographical coordinate of Toronto are 43.653963, -79.387207.


Get the *Foursquare API* credentials.

In [54]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: QBOJ201MYL2VGDGB2IWLVQORWVDJDY2UTKOUFE33YMB25EPE
CLIENT_SECRET:EOSHKOQRBEVLEQGUF1ELZ11QGPTNHENVPOYPIT1FR2GBNA3C


Let's start by making a map of Toronto with popups for each of our neighborhoods.

In [19]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.0-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   1.07 MB/s
branca-0.3.0-p 100% |################################| Time: 0:00:00  30.33 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  34.75 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  41.76 MB/s


In [29]:
map_toronto = folium.Map(location = [lat, lng], zoom_start = 10)

for ilat, ilng, iborough, inborhood in zip(df_toronto["Latitude"], df_toronto["Longitude"], df_toronto["Borough"], df_toronto["Neighborhood"]):
    lbl = "{}, {}".format(inborhood, iborough)
    lbl = folium.Popup(lbl, parse_html = True)
    folium.CircleMarker([ilat, ilng],\
                        radius = 5,\
                        popup = lbl,\
                        color = "blue",\
                        fill = True,\
                        fill_color = "#3186cc",\
                        fill_opacity = 0.7,\
                        parse_html = False).add_to(map_toronto)

map_toronto

Copy the function to get nearby venues for each of the neighborhoods (don't forget to define LIMIT).

In [24]:
limit = 100

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
toronto_venues = getNearbyVenues(names = df_toronto["Neighborhood"], \
                                latitudes = df_toronto["Latitude"], \
                                longitudes = df_toronto["Longitude"])
print("Finish")

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [32]:
print(toronto_venues.shape)
toronto_venues.head()

(1707, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Beaches Fitness,43.680319,-79.290991,Gym / Fitness Center


First check how many venues are returned for each neighborhood.

In [33]:
toronto_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,54,54,54,54,54,54
"Brockton, Exhibition Place, Parkdale Village",24,24,24,24,24,24
Business reply mail Processing Centre969 Eastern,19,19,19,19,19,19
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,80,80,80,80,80,80
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


Check how many different categories there are.

In [34]:
print("There are {} categories.".format(len(toronto_venues["Venue Category"].unique())))

There are 232 categories.


Before we cluster the dataset, let's rearrange the dataset using one-hot encoding such that each 'Venue Category' becomes a column variable.

In [35]:
toronto_onehot = pd.get_dummies(toronto_venues[["Venue Category"]], prefix = "", prefix_sep = " ")
toronto_onehot["Neighborhood"] = toronto_venues["Neighborhood"] # to add the Neighborhood column back to the data frame (in last position)
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns] # to switch the Neighborhood column back to first position
toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
toronto_onehot.shape

(1707, 233)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [39]:
toronto_grouped = toronto_onehot.groupby("Neighborhood").mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.01,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.01,0.000000
1,Berczy Park,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.000000
3,Business reply mail Processing Centre969 Eastern,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.052632
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.00,0.0,0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.000000


In [40]:
toronto_grouped.shape

(38, 233)

Define a funtion to sort the most occurring venues for a given record in descending order.

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Use the method to create a new dataframe that contains the top 10 venues for each neighborhood.

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Gym,Bar,Hotel,Restaurant,Breakfast Spot
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Beer Bar,Steakhouse,Cheese Shop,Café,Seafood Restaurant,Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Performing Arts Venue,Nightclub,Café,Breakfast Spot,Italian Restaurant,Convenience Store,Pet Store,Gym / Fitness Center,Climbing Gym
3,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Pizza Place,Recording Studio,Butcher,Restaurant,Burrito Place,Brewery,Skate Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Plane,Sculpture Garden,Boutique,Boat or Ferry,Harbor / Marina,Airport Gate,Airport


In [43]:
neighborhoods_venues_sorted.shape

(38, 11)

## Cluster neighborhoods based on the top 10 venues

In [44]:
from sklearn.cluster import KMeans

Divide the dataset into 5 clusters.

In [1]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop("Neighborhood", 1)
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10]

NameError: name 'toronto_grouped' is not defined

Create a dataframe that includes the clusters as well as the top 10 venues.

In [50]:
toronto_merged = df_toronto
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,The Beaches,East Toronto,43.676357,-79.293031,2,Gym / Fitness Center,Neighborhood,Coffee Shop,Other Great Outdoors,Pub,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,M4K,"The Danforth West, Riverdale",East Toronto,43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Fruit & Vegetable Store,Pizza Place,Lounge,Juice Bar
2,M4L,"The Beaches West, India Bazaar",East Toronto,43.668999,-79.315572,2,Park,Hotel,Liquor Store,Sandwich Place,Fast Food Restaurant,Burger Joint,Fish & Chips Shop,Burrito Place,Steakhouse,Italian Restaurant
3,M4M,Studio District,East Toronto,43.659526,-79.340923,1,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Park,Brewery,Seafood Restaurant
4,M4N,Lawrence Park,Central Toronto,43.728020,-79.388790,2,Bus Line,Park,Swim School,Dim Sum Restaurant,Yoga Studio,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


Let's visualize the clusters at last.

In [52]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [53]:
# create map
map_clusters = folium.Map(location=[lat, lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters